In [1]:
import numpy as np
import pandas as pd 
import plotly.express as px

In [2]:
SEED = 42

## Carga de datos

In [3]:
df = pd.read_csv('/kaggle/input/car-price-dataset/car_price_dataset.csv')
df.sample(5, random_state=SEED)

,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
6252,BMW,3 Series,2005,1.2,Diesel,Manual,257760,5,5,2000
4684,Volkswagen,Golf,2013,2.9,Hybrid,Automatic,111790,5,3,11164
1731,Toyota,RAV4,2011,5.0,Hybrid,Automatic,13473,5,4,14630
4742,Mercedes,E-Class,2007,3.8,Diesel,Manual,133298,4,5,7334
4521,Honda,Civic,2012,1.3,Petrol,Automatic,18611,3,5,10127


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         10000 non-null  object 
 1   Model         10000 non-null  object 
 2   Year          10000 non-null  int64  
 3   Engine_Size   10000 non-null  float64
 4   Fuel_Type     10000 non-null  object 
 5   Transmission  10000 non-null  object 
 6   Mileage       10000 non-null  int64  
 7   Doors         10000 non-null  int64  
 8   Owner_Count   10000 non-null  int64  
 9   Price         10000 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 781.4+ KB


In [5]:
df.describe()

,Year,Engine_Size,Mileage,Doors,Owner_Count,Price
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,2011.543700,3.000560,149239.111800,3.497100,2.991100,8852.96440
std,6.897699,1.149324,86322.348957,1.110097,1.422682,3112.59681
min,2000.000000,1.000000,25.000000,2.000000,1.000000,2000.00000
25%,2006.000000,2.000000,74649.250000,3.000000,2.000000,6646.00000
50%,2012.000000,3.000000,149587.000000,3.000000,3.000000,8858.50000
75%,2017.000000,4.000000,223577.500000,4.000000,4.000000,11086.50000
max,2023.000000,5.000000,299947.000000,5.000000,5.000000,18301.00000


- **Year** -> año de producción. El registro va desde el año 2000 hasta el 2023
- **Engine_Size** -> tamaño del motor en Litros. Tenemos que ver si tratarla como variable numerica o categórica, ya que no es un valor continuo como tal.
- **Mileage** -> millas/cuenta kilometros. La desviación estándar es muy alta y fijandonos en los valores max y min, confirmamos la disparidad entre coches muy usados y apenas sacados del garaje. Sin duda, sera una variable clave para determinar el precio.
- **Doors** -> número de puertas. De 1 hasta 5 (el maletero cuenta como puerta). Vemos que la mitad de los registros (q50%) tiene 3 puertas, lo que indica que son modelos de coches compactos
- **Owner_Count** -> número de propietarios. Desde 1 hasta 5, siendo 3 la media (habría que ver la moda)
- **Price** -> es variable objetivo, no se indica tipo de moneda, así por simpleza lo denominaremos como "unidades monetarias"

In [6]:
df['Engine_Size'].unique()

array([4.2, 2. , 2.6, 2.7, 3.4, 4.7, 3.1, 1.3, 4.5, 4.8, 2.3, 2.4, 3.2,
       2.9, 1.5, 1.8, 4.6, 2.8, 3.9, 3.3, 1.9, 3.5, 1.4, 2.2, 1. , 1.2,
       4.9, 3.6, 3.8, 4. , 1.7, 1.6, 4.1, 1.1, 2.5, 3. , 5. , 2.1, 3.7,
       4.3, 4.4])

Vemos que no son valores exactos por lo que podríamos discretizar la variable en el procesamiento y tratarla como categorica.

**NOTA** Al reagrupar podemos los modelos lineales pueden discernir mejor

In [7]:
df['Owner_Count'].value_counts()

Owner_Count
5    2036
1    2036
2    2020
3    1977
4    1931
Name: count, dtype: int64

Tiene una distribución muy equilibrada, siendo lo más común los "extremos": o bien solo 1 propietario o bien 5

In [8]:
df.describe(include="object")

,Brand,Model,Fuel_Type,Transmission
count,10000,10000,10000,10000
unique,10,30,4,3
top,Ford,Accord,Electric,Manual
freq,1048,365,2625,3372


De un primer vistazo vemos los valores únicos para cada variable, con las visualizaciones veremos mejor sus distribuciones. También podemos usar el método `value_counts` si no queremos crear gráficas. 

**NOTA** Estas variables las podemos tratar como `category` por su naturaleza.

In [9]:
cat_cols = df.select_dtypes(include="object").columns
df[cat_cols] = df[cat_cols].astype('category')

In [10]:
df.isna().sum() / df.shape[0]

Brand           0.0
Model           0.0
Year            0.0
Engine_Size     0.0
Fuel_Type       0.0
Transmission    0.0
Mileage         0.0
Doors           0.0
Owner_Count     0.0
Price           0.0
dtype: float64

In [11]:
df.duplicated().sum()

0

No hay valores perdidos ni tampoco duplicados, por lo que no será necesario realizar utilizar técnicas complejas en el procesamiento de los datos

## Visualización de los datos